In [ ]:
#--------------------------------------------------------------------------------
# These tokens are needed for user authentication.
# Credentials can be generates via Twitter's Application Management:
#	https://apps.twitter.com/app/new
#--------------------------------------------------------------------------------

consumer_key = "4OlkTqJmjaldPXW3Aqe6RUTo0"
consumer_secret = "Rujs81iiCgw1u0ZbBMZCQneImazJA8wnImqWbn5wGgpvgftKAg"
access_key = "1119890635279015936-4euB1D3u1O9EKyI5kuPSfChTC8LoTQ"
access_secret = "gsp51xMjCvBlSt7kB3s1eNxSfqhRqozYwefZaAW9V5msE"
max_keywords = 30

In [ ]:

from twitter import *

import time
import json
import sys
import os
import re
sys.path.append(".")

def file_under_dir(directory_name,filename):
    directory_name = os.getcwd() + directory_name
    filepath = os.path.join(directory_name, filename)
    try: 
        os.makedirs(directory_name,0o777)
    except OSError:
        if not os.path.isdir(directory_name):
            raise
    return filepath

#-----------------------------------------------------------------------
# create twitter API object
#-----------------------------------------------------------------------
api = Api(consumer_key,consumer_secret,access_key,access_secret)


def divide_chunks(l, n): 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 

total_lexicon = []
with open('lexicon/twitter_lexicon') as f:
    for line in f:
        for word in line.split():
           total_lexicon.append(word)
   

# perform a basic search

# Twitter API docs:
# https://dev.twitter.com/rest/reference/get/search/tweets
#-----------------------------------------------------------------------
while(1):
    for sublexicon in list(divide_chunks(total_lexicon,max_keywords-1 )):
        str_sublexicon = ""
        str_sublexicon = ' OR '.join(sublexicon)
        results = []
        try:
            results = api.GetSearch(
                term=str_sublexicon ,lang = "en")
        except error.TwitterError as err:
            print(err.message[0]['code'])
               
            if (err.message[0]['code'] == 88):
                print ("Rate Limit Exceeded, Sleep for 15 Mins") 
                for i in range(15,0,-1):
                    sys.stdout.write(str(i)+' ')
                    sys.stdout.flush()
                    time.sleep(60)
                continue
        for tweet in results:
            with open(file_under_dir('/tweets/','tweet_' + tweet.id_str +'.json'), 'a', encoding='utf8') as file:
                json.dump(tweet._json,file,indent=4,sort_keys=True)

